In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
from local.load import *

In [14]:
bs = 4
letters = list(string.ascii_lowercase)

In [15]:
t = [(1, (2,3)), (1, (2,3))]

In [16]:
fa_collate(t)
test_eq(fa_collate(t), default_collate(t))
test_eq(L(fa_collate(t)).map(type), [Tensor, tuple])

t = [(1,(2,(3,4))),(1,(2,(3,4)))]
test_eq(fa_collate(t), default_collate(t))
test_eq(L(fa_collate(t)).map(type), [Tensor, tuple])
test_eq(L(fa_collate(t)[1]).map(type), [Tensor, tuple])

In [17]:
t0 = np.array([1,2])
t = [t0, (t0, t0)]

test_eq(fa_convert(t), default_convert(t))
test_eq(L(fa_convert(t)).map(type), [Tensor, tuple])

In [18]:
class RandDL(DataLoader):
    def create_item(self, s):
        r = random.random()
        return r if r<0.95 else stop()
    
L(RandDL())

(#18) [0.9031972110613757,0.004867683068232931,0.09051894413740413,0.34825489509882845,0.05061279830630383,0.31301221184499706,0.9494824322759067,0.1731067409489636,0.8046956999844335,0.45710757366722365...]

In [67]:
L(RandDL(bs=4, num_workers=4, drop_last=True)).map(len)

(#10) [4,4,4,4,4,4,4,4,4,4]

In [21]:
def _rand_item(s):
    r = random.random()
    return r if r<0.95 else stop()

L(DataLoader(create_item=_rand_item))

(#2) [0.5921194925602398,0.41875242864255846]

In [22]:
ds1 = DataLoader(letters)
test_eq(L(ds1), letters)
test_eq(len(ds1), 26)

test_shuffled(L(DataLoader(letters, shuffle=True)), letters)

ds1 = DataLoader(letters, indexed=False)
test_eq(L(ds1), letters)
test_eq(len(ds1), 26)

t2 = L(tensor([0,1,2]),tensor([3,4,5]))
ds2 = DataLoader(t2)
test_eq_type(L(ds2), t2)

t3 = L(np.array([0,1,2]), np.array([3,4,5]))
ds3 = DataLoader(t3)
test_eq_type(L(ds3), t3)

ds4 = DataLoader(t3, create_batch=noops, after_iter=lambda: setattr(t3, 'f', 1))
test_eq_type(L(ds4), t3)
test_eq(t3.f, 1)

In [57]:
def twoepochs(d): return " ".join("".join(o) for _ in range(2) for o in d)

In [58]:
ds1 = DataLoader(letters, bs=4, drop_last=True, num_workers=0)
test_eq(twoepochs(ds1), 'abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx')

ds1 = DataLoader(letters, bs=4, drop_last=True, num_workers=0, n=5)
test_eq(twoepochs(ds1), 'abcd abcd')

ds1 = DataLoader(letters,4,num_workers=2)
test_eq(twoepochs(ds1), 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')

ds1 = DataLoader(range(12), bs=4, num_workers=3)
test_eq_type(L(ds1), L(tensor([0,1,2,3]),tensor([4,5,6,7]),tensor([8,9,10,11])))

ds1 = DataLoader([str(i) for i in range(11)], bs=4, after_iter=lambda: setattr(t3, 'f', 2))
test_eq_type(L(ds1), L(['0','1','2','3'],['4','5','6','7'],['8','9','10']))
test_eq(t3.f, 2)

it = iter(DataLoader(map(noop,range(20)), bs=4, num_workers=1))
test_eq_type([next(it) for _ in range(3)], [tensor([0,1,2,3]),tensor([4,5,6,7]),tensor([8,9,10,11])])

In [76]:
class SleepyQueue():
    "Simulate a queue with varying latency"
    def __init__(self, q): self.q=q
    def __iter__(self):
        while True:
            time.sleep(random.random()/100)
            try: yield self.q.get_nowait()
            except queues.Empty: return
            
q = Queue()
for o in range(30): q.put(o)
it = SleepyQueue(q)

%time test_shuffled(L(DataLoader(it, num_workers=4)), range(30))

CPU times: user 6.7 ms, sys: 16.2 ms, total: 22.9 ms
Wall time: 97.6 ms


In [77]:
class A(TensorBase): pass

for nw in (0,2):
    t = A(tensor([1,2]))
    dl = DataLoader([t,t,t,t,t,t,t,t], bs=4, num_workers=nw)
    b = next(iter(dl))
    test_eq(type(b), A)

    t = (A(tensor([1,2])),)
    dl = DataLoader([t,t,t,t,t,t,t,t], bs=4, num_workers=nw)
    b = next(iter(dl))
    test_eq(type(b[0]), A)

In [78]:
class A(TensorBase): pass
t = A(tensor(1,2))

tdl = DataLoader([t,t,t,t,t,t,t,t], bs=4, num_workers=2, after_batch=to_device)
b = next(iter(tdl))
test_eq(type(b), A)

# Unknown attributes are delegated to `dataset`
test_eq(tdl.pop(), tensor(1,2))

In [79]:
tdl.pop()

tensor([1, 2])